# Jigsaw Unintended Bias in Toxicity Classification

#### Since the data we have contains sequence pattren, Here in this file I am trying the same problem with using bidirectional LSTM and GRU, which typically perfroms great on sequential data.

In [1]:
#For Data loading and preprocessing
import pandas as pd

#For matrix operations
import numpy as np

#For Ploting
import matplotlib.pyplot as plt
%matplotlib inline


from sklearn.feature_extraction.text import TfidfVectorizer

#Importing all keras necessary modules
import keras
from keras.models import Sequential,Model,load_model
from keras.layers import Dense,Dropout
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Bidirectional,add,GRU,Input,concatenate,GlobalMaxPooling1D,GlobalAveragePooling1D,SpatialDropout1D,CuDNNLSTM
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.optimizers import *

##To see the progress of the iterations
from tqdm import tqdm

#For splitting the data
from sklearn.model_selection import train_test_split

#Performance metrices
from sklearn import metrics
from sklearn.metrics import roc_auc_score,roc_curve,auc,log_loss,confusion_matrix,make_scorer

#For ignoring deprication warnings
import warnings
warnings.filterwarnings("ignore")
import keras.backend as K
from keras.models import load_model

Using TensorFlow backend.


In [0]:
#To read the file form google-drive
#Ref: https://www.youtube.com/watch?v=oqMImCeXi6o
#!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
#Getting authorization from drive to read the file
#Ref: https://www.youtube.com/watch?v=oqMImCeXi6o
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':'1bVz4c1W16bTmEWThVuVQ9haktmlCXn4i'})
downloaded.GetContentFile('data_processed.csv')

In [7]:
#Getting all preprocessed data
data = pd.read_csv("data_processed.csv")
data.head()

,target,severe_toxicity,obscene,threat,insult,identity_attack,sexual_explicit,male,female,homosexual_gay_or_lesbian,christian,jewish,muslim,black,white,psychiatric_or_mental_illness,preprocessed_reviews,word_count,comment_length,num_bad_words,len_bad_words,profinity_frequency,profinity_len_frequency,profinity_mul,profinity_sum,profinity_HM
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,cool like would want mother read really great ...,11,60,0,0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,thank would make life lot less anxiety inducin...,14,75,0,0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,urgent design problem kudos taking impressive,6,45,0,0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,something able install site releasing,5,37,0,0,0.0,0.0,0.0,0.0,0.0
4,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,haha guys bunch losers,4,22,0,0,0.0,0.0,0.0,0.0,0.0


In [0]:
#For appling DL algorithms we do not require hand crafted features (of course that's the beauty of DL)
X = data['preprocessed_reviews'] #Hence taking only preprocessed text as input


#Taking all output labels  
Y = data[['target',
 'severe_toxicity',
 'obscene',
 'threat',
 'insult',
 'identity_attack',
 'sexual_explicit',
 'male',
 'female',
 'homosexual_gay_or_lesbian',
 'christian',
 'jewish',
 'muslim',
 'black',
 'white',
 'psychiatric_or_mental_illness']]

#Spliting the data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42,shuffle=False)

In [9]:
ALL_labels= Y.columns.to_list() # list of all output label names  
ALL_labels

['target',
 'severe_toxicity',
 'obscene',
 'threat',
 'insult',
 'identity_attack',
 'sexual_explicit',
 'male',
 'female',
 'homosexual_gay_or_lesbian',
 'christian',
 'jewish',
 'muslim',
 'black',
 'white',
 'psychiatric_or_mental_illness']

In [0]:
def get_aux_labels(df):
    return df[['target']],df[['severe_toxicity','obscene','threat','insult','identity_attack',
 'sexual_explicit']],df[['male','female','homosexual_gay_or_lesbian','christian','jewish','muslim','black','white','psychiatric_or_mental_illness']]

In [0]:
y_train_target,y_train_aux_target,y_train_identity = get_aux_labels(y_train)

y_test_target,y_test_aux_target,y_test_identity = get_aux_labels(y_test)

In [0]:
# Defing own AUC based metric,specially mentioned for this competition
# This code was inspired from :https://www.kaggle.com/kenkrige/subgroup-analysis

def AUC(df_y):

    fpr, tpr, thresholds = roc_curve(df_y["actual"],df_y["predict"],pos_label=1)
    return auc(fpr, tpr)

def Mp(data, p=-5.0):
    return np.average(data.astype(float) ** p) ** (1/p)


def final_AUC_metric(y_true, y_predicted,final_AUC_metric_alone=True):
        
    IDENTITY_labels = ["male",
                       "female",
                       "homosexual_gay_or_lesbian",
                       "christian",
                       "jewish",
                       "muslim",
                       "black",
                       "white",
                       "psychiatric_or_mental_illness"]
    y_true= pd.DataFrame(y_true, columns=ALL_labels).reset_index(drop=True)  #Converting Actual output arrays to dataframes with same labels  
    y_predicted= pd.DataFrame(y_predicted, columns=ALL_labels).reset_index(drop=True) #Converting Predicted output arrays to dataframes with same labels  
    
    y_df = pd.DataFrame(y_true['target'].to_list(), columns=["actual"]) #putting only ['target'] values of both "y_true" and "y_predicted" into single DF
    y_df["predict"]=y_predicted['target'].to_list()
    

    
    results = pd.DataFrame(columns = ['SUB', 'BPSN', 'BNSP'], index = IDENTITY_labels) # Created to collect all AUCs
    
    overall = AUC(df_y=y_df)  #This overall AUC (diretly between (y_true['target'],y_predicted['target']  )
    
    for label in IDENTITY_labels: # Calculating BPSN and BNSP for each identity label 
        y_true[label] = y_true[label] == 1
        y_true['target'] = y_true['target']==1
        
        bpsn = y_true[label]^y_true['target']    
        bnsp = ~(y_true[label]^y_true['target'])
        
        results.loc[label,'BPSN']= AUC(df_y = y_df[bpsn]) 
        results.loc[label,'BNSP']= AUC(df_y = y_df[bnsp])
        results.loc[label,'SUB']=  AUC(df_y = y_df[y_true[label]])

    results.loc['Mp',:] = results.apply(Mp, axis= 0)

    

    final_auc = ((results.loc['Mp',:].sum()) + overall) / 4

    if final_AUC_metric_alone:
        return final_auc
    else:
        return final_auc, results
#final_AUC = make_scorer(final_AUC_metric, greater_is_better=True,final_AUC_metric_alone=True)

In [0]:
#Ref: https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
tokens = Tokenizer()
tokens.fit_on_texts(X_train)
vocab_size = len(tokens.word_index) + 1
# integer encode the documents
X_train_ints = tokens.texts_to_sequences(X_train)

X_test_ints = tokens.texts_to_sequences(X_test)

max_length = 500 #Setting equal length of 500 for every comment_text
X_train_ints = pad_sequences(X_train_ints, maxlen=max_length, padding='post')

X_test_ints = pad_sequences(X_test_ints, maxlen=max_length, padding='post')

In [0]:
#Getting 'glove.6B.100d.txt' file 
downloaded = drive.CreateFile({'id':'11hKZPIxqRbmisbmfUW36VM8RNzfkYkYF'}) # replace the id with id of file you want to access
downloaded.GetContentFile('glove.6B.100d.txt') 

In [15]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [16]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tqdm(tokens.word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 217265/217265 [00:00<00:00, 585610.08it/s]


### 1. Bi-directional GRU

In [0]:

input_s = Input(shape=(max_length,), dtype='int32', name='input_s')

GRUs = Embedding(input_dim=vocab_size,input_length=max_length,weights=[embedding_matrix],trainable=False,output_dim=embedding_matrix.shape[1])(input_s)

GRUs = SpatialDropout1D(0.3)(GRUs)

GRUs = Bidirectional(GRU(128, return_sequences=True))(GRUs)
GRUs = Bidirectional(GRU(128, return_sequences=True))(GRUs)
hidden1 = concatenate([GlobalMaxPooling1D()(GRUs),GlobalAveragePooling1D()(GRUs) ])

hidden2 = add([hidden1, Dense(512, activation='relu')(hidden1)])
hidden3 = add([hidden2, Dense(512, activation='relu')(hidden2)])

#Multi_Outputs
result = Dense(1, activation='sigmoid',name='result')(hidden3) #target
aux_result = Dense(y_train_aux_target.shape[1], activation='sigmoid',name='aux_result')(hidden3) #
identity_result = Dense(y_train_identity.shape[1], activation='sigmoid',name='identity_result')(hidden3)

model = Model(inputs=input_s, outputs=[result, aux_result,identity_result])
adam = Adam(lr= 0.0006)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_s (InputLayer)            (None, 500)          0                                            
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, 500, 100)     21726600    input_s[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_12 (SpatialDr (None, 500, 100)     0           embedding_12[0][0]               
__________________________________________________________________________________________________
bidirectional_21 (Bidirectional (None, 500, 256)     175872      spatial_dropout1d_12[0][0]       
__________________________________________________________________________________________________
bidirectio

In [0]:
history = model.fit(
{'input_s': X_train_ints},
{'result': y_train_target,
    'aux_result':y_train_aux_target,
    'identity_result':y_train_identity},
batch_size=512,
epochs=4,
validation_data=[{'input_s': X_test_ints},
                  {'result': y_test_target,
          'aux_result':y_test_aux_target,
          'identity_result':y_test_identity}])

Train on 1249618 samples, validate on 535551 samples
Epoch 1/4
 298496/1249618 [======>.......................] - ETA: 6:51:40 - loss: 0.2906 - result_loss: 0.1944 - aux_result_loss: 0.0451 - identity_result_loss: 0.0511 - result_acc: 0.9322 - aux_result_acc: 0.9876 - identity_result_acc: 0.9861

In [0]:
#History of fitting
pd.DataFrame(history.history)

,aux_result_acc,aux_result_loss,identity_result_acc,identity_result_loss,loss,result_acc,result_loss,val_aux_result_acc,val_aux_result_loss,val_identity_result_acc,val_identity_result_loss,val_loss,val_result_acc,val_result_loss
0,0.989538,0.034499,0.992251,0.024733,0.225061,0.939478,0.165829,0.989867,0.030094,0.996385,0.010376,0.193054,0.941003,0.152584
1,0.990756,0.027901,0.996242,0.010442,0.181899,0.945788,0.143556,0.990254,0.028192,0.996741,0.009067,0.182248,0.943636,0.144990
2,0.991033,0.026496,0.996598,0.009376,0.173214,0.947805,0.137342,0.989904,0.028607,0.996760,0.008939,0.185810,0.941750,0.148265
3,0.991199,0.025709,0.996747,0.008941,0.168440,0.948536,0.133790,0.990508,0.027050,0.996938,0.008527,0.177589,0.944000,0.142012


In [0]:
downloaded = drive.CreateFile({'id':'1OYzd8cAlefRcDPM3TGc9zosvGDwYbYei'})
downloaded.GetContentFile('GRU_128_model.h5')

In [18]:
#model.save('GRU_128_model.h5')
model_GRU = load_model('GRU_128_model.h5')





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Evaluating the GRU Model

In [0]:
prediction_train_GRU = model_GRU.predict(X_train_ints,batch_size=1024)


results_train_GRU   = final_AUC_metric(y_true=y_train, y_predicted=np.hstack((prediction_train_GRU[0],prediction_train_GRU[1],prediction_train_GRU[2]))
                          ,final_AUC_metric_alone=False)

In [20]:
print("Individual AUCs for all identity lables in train data.")
print("-----------------------------------------------------------")
results_train_GRU[1]

Individual AUCs for all identity lables in train data.
-----------------------------------------------------------


,SUB,BPSN,BNSP
male,0.919915,0.899428,0.970506
female,0.920779,0.903863,0.969247
homosexual_gay_or_lesbian,0.839443,0.803015,0.974197
christian,0.928425,0.940823,0.951624
jewish,0.891134,0.870905,0.969504
muslim,0.858035,0.822108,0.97438
black,0.837163,0.787605,0.976653
white,0.851046,0.79152,0.978752
psychiatric_or_mental_illness,0.924017,0.893697,0.972906
Mp,0.880937,0.847051,0.970687


In [21]:
print("Final AUC value for Train data:")
results_train_GRU[0]

Final AUC value for Train data:


0.9143455883287539

In [0]:
prediction_test_GRU = model_GRU.predict(X_test_ints,batch_size=1024)


results_test_GRU   = final_AUC_metric(y_true=y_test, y_predicted=np.hstack((prediction_test_GRU[0],prediction_test_GRU[1],prediction_test_GRU[2]))
                          ,final_AUC_metric_alone=False)

In [23]:
print("Individual AUCs for all identity lables in Test data.")
print("-----------------------------------------------------------")
results_test_GRU[1]

Individual AUCs for all identity lables in Test data.
-----------------------------------------------------------


,SUB,BPSN,BNSP
male,0.90492,0.883349,0.964173
female,0.904801,0.893481,0.959032
homosexual_gay_or_lesbian,0.824629,0.792441,0.968468
christian,0.921393,0.931261,0.944542
jewish,0.872002,0.844679,0.964684
muslim,0.855626,0.8085,0.97034
black,0.822899,0.755966,0.975351
white,0.833,0.759401,0.976705
psychiatric_or_mental_illness,0.882848,0.858177,0.962593
Mp,0.864885,0.824491,0.964836


In [24]:
print("Final AUC value for Test data:")
results_test_GRU[0]

Final AUC value for Test data:


0.9011745406654919

### 2. Bidirectional-LSTM

In [0]:
input_s = Input(shape=(max_length,), dtype='int32', name='input_s')

LSTMs = Embedding(input_dim=vocab_size,input_length=max_length,weights=[embedding_matrix],trainable=False,output_dim=embedding_matrix.shape[1])(input_s)

LSTMs = SpatialDropout1D(0.3)(LSTMs)

LSTMs = Bidirectional(CuDNNLSTM(128, return_sequences=True))(LSTMs)
LSTMs = Bidirectional(CuDNNLSTM(128, return_sequences=True))(LSTMs)
hidden1 = concatenate([GlobalMaxPooling1D()(LSTMs),GlobalAveragePooling1D()(LSTMs)])

hidden2 = add([hidden1, Dense(512, activation='relu')(hidden1)])
hidden3 = add([hidden2, Dense(512, activation='relu')(hidden2)])

#Multi_Outputs
result = Dense(1, activation='sigmoid',name='result')(hidden3) #target
aux_result = Dense(y_train_aux_target.shape[1], activation='sigmoid',name='aux_result')(hidden3) #
identity_result = Dense(y_train_identity.shape[1], activation='sigmoid',name='identity_result')(hidden3)

model = Model(inputs=input_s, outputs=[result, aux_result,identity_result])
adam = Adam(lr= 0.0006)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
print(model.summary())

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_s (InputLayer)            (None, 500)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 500, 100)     21726600    input_s[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_2 (SpatialDro (None, 500, 100)     0           embedding_2[0][0]                
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 500, 256)     235520      spatial_dropout1d_2[0][0]        
____________________________________________________________________________________________

In [0]:
history2 = model.fit(
{'input_s': X_train_ints},
{'result': y_train_target,
    'aux_result':y_train_aux_target,
    'identity_result':y_train_identity},
batch_size=512,
epochs=4,
validation_data=[{'input_s': X_test_ints},
                  {'result': y_test_target,
          'aux_result':y_test_aux_target,
          'identity_result':y_test_identity}])

Train on 1249618 samples, validate on 535551 samples
Epoch 1/4
1249618/1249618 [==============================] - 3047s 2ms/step - loss: 0.1813 - result_loss: 0.1426 - aux_result_loss: 0.0279 - identity_result_loss: 0.0107 - result_acc: 0.9460 - aux_result_acc: 0.9907 - identity_result_acc: 0.9961 - val_loss: 0.1811 - val_result_loss: 0.1443 - val_aux_result_loss: 0.0279 - val_identity_result_loss: 0.0088 - val_result_acc: 0.9434 - val_aux_result_acc: 0.9903 - val_identity_result_acc: 0.9968
Epoch 2/4
1249618/1249618 [==============================] - 3050s 2ms/step - loss: 0.1724 - result_loss: 0.1368 - aux_result_loss: 0.0264 - identity_result_loss: 0.0092 - result_acc: 0.9477 - aux_result_acc: 0.9910 - identity_result_acc: 0.9966 - val_loss: 0.1801 - val_result_loss: 0.1439 - val_aux_result_loss: 0.0277 - val_identity_result_loss: 0.0085 - val_result_acc: 0.9432 - val_aux_result_acc: 0.9902 - val_identity_result_acc: 0.9969
Epoch 3/4
1249618/1249618 [==============================] 

In [0]:
pd.DataFrame(history2.history)

,val_loss,val_result_loss,val_aux_result_loss,val_identity_result_loss,val_result_acc,val_aux_result_acc,val_identity_result_acc,loss,result_loss,aux_result_loss,identity_result_loss,result_acc,aux_result_acc,identity_result_acc
0,0.181065,0.144323,0.027929,0.008814,0.943376,0.990332,0.996846,0.181293,0.142645,0.027932,0.010716,0.946046,0.990726,0.996118
1,0.180093,0.143864,0.027730,0.008500,0.943221,0.990188,0.996912,0.172393,0.136766,0.026383,0.009244,0.947725,0.991018,0.996632
2,0.173211,0.138562,0.026483,0.008166,0.945553,0.990601,0.997029,0.166786,0.132715,0.025423,0.008649,0.948801,0.991216,0.996832
3,0.175894,0.140708,0.026990,0.008196,0.944368,0.990302,0.996942,0.163348,0.130132,0.024850,0.008366,0.949770,0.991329,0.996926


In [0]:
downloaded = drive.CreateFile({'id':'1CH5hrfHfMXkmZykXTy1CCmuk23mTRGzE'})
downloaded.GetContentFile('LSTM_128_model.h5')

In [0]:
#model.save('LSTM_128_model.h5')
model_LSTM = load_model('LSTM_128_model.h5')

### Evaluating the LSTM model

In [0]:
prediction_test_LSTM = model_LSTM.predict(X_test_ints,batch_size=1024)

results_test_LSTM = final_AUC_metric(y_true=y_test, y_predicted=np.hstack((prediction_test_LSTM[0],prediction_test_LSTM[1],prediction_test_LSTM[2]))
                          ,final_AUC_metric_alone=False)

In [0]:
prediction_train_LSTM = model_LSTM.predict(X_test_ints,batch_size=1024)

results_train_LSTM = final_AUC_metric(y_true=y_test, y_predicted=np.hstack((prediction_train_LSTM[0],prediction_train_LSTM[1],prediction_train_LSTM[2]))
                          ,final_AUC_metric_alone=False)

In [29]:
print("Individual AUCs for all identity lables in train data")
print("-----------------------------------------------------------")
results_train_LSTM[1]

Individual AUCs for all identity lables in train data
-----------------------------------------------------------


,SUB,BPSN,BNSP
male,0.909608,0.913021,0.953612
female,0.909311,0.929136,0.941677
homosexual_gay_or_lesbian,0.833091,0.815263,0.96553
christian,0.925166,0.944067,0.937236
jewish,0.871444,0.906045,0.936999
muslim,0.857696,0.866571,0.954591
black,0.835007,0.80147,0.969378
white,0.837532,0.813361,0.967609
psychiatric_or_mental_illness,0.890834,0.893438,0.953436
Mp,0.870565,0.866693,0.952895


In [30]:
print("Final AUC value on train data:")
results_train_LSTM[0]

Final AUC value on train data:


0.9106687690024847

In [31]:
print("Individual AUCs for all identity lables in test data.")
print("-----------------------------------------------------------")
results_test_LSTM[1]

Individual AUCs for all identity lables in test data.
-----------------------------------------------------------


,SUB,BPSN,BNSP
male,0.909608,0.913021,0.953612
female,0.909311,0.929136,0.941677
homosexual_gay_or_lesbian,0.833091,0.815263,0.96553
christian,0.925166,0.944067,0.937236
jewish,0.871444,0.906045,0.936999
muslim,0.857696,0.866571,0.954591
black,0.835007,0.80147,0.969378
white,0.837532,0.813361,0.967609
psychiatric_or_mental_illness,0.890834,0.893438,0.953436
Mp,0.870565,0.866693,0.952895


In [32]:
print("Final AUC value on Test data:")
results_test_LSTM[0]

Final AUC value on Test data:


0.9106687690024847

### Results

In [33]:
pd.DataFrame(zip((results_test_LSTM[0],results_train_LSTM[0]),(results_test_GRU[0],results_train_GRU[0])),
             columns=['Test_Toxic_AUC','Train_Toxic_AUC'],index=['LSTM','GRU'])

,Test_Toxic_AUC,Train_Toxic_AUC
LSTM,0.910669,0.901175
GRU,0.910669,0.914346


#### This result shows that using Bi-directional LSTM/GRU, even though the comment contains the identity labels,model has no confusion to determine whether the comment is toxic or not with 91.066% of chance.